In [21]:
import numpy as np
from numpy.linalg import det
# from sympy import Matrix, symbols, EmptySet
# from sympy.solvers.solveset import linsolve
from copy import copy
from cvxopt.glpk import ilp
from cvxopt import matrix


def get_hermit(dimensions=6, max_minor = 5):
    diag = np.array([1]*(dimensions)).astype('d')
    diag[-1] = max_minor
    pre_last = np.random.randint(max_minor, size=dimensions - 1)
    hermit = np.diag(diag)
    hermit[dimensions-1][:-1] = pre_last
    return hermit


def apply_restrictor(hermit, dimensions=6, max_minor = 5):
    continue_flag = True
    restricted = 0
    while continue_flag:
        restricted = np.append(hermit, np.random.randint(-max_minor, max_minor + 1, size=(1, dimensions)), 0)
        continue_flag = not is_valid(restricted)
    return restricted


def minor(arr, i=0):
#     ith row removed
    return arr[np.array(range(i)+range(i+1, arr.shape[0]))[:, np.newaxis],
               np.array(range(arr.shape[1]))]


def is_valid(A):
    return all(det(minor(A,i)) for i in xrange(A.shape[0]))


# prime = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


# def generate_b(dimensions=5):
#     """finish it"""
#     max_minor = dimensions - 1
#     b = np.zeros(shape=(dimensions+1,1), dtype=float)
#     for i in xrange(-max_minor,max_minor+1):
#         b[-2] = i
#         for j in xrange(-10,10+1):
#             b[-1] = j
#             yield b.astype(float)
            
            
def is_valid_solution(solution):
#     return not isinstance(s, EmptySet) and all(x for x in solution)
    return  any(x for x in solution)

In [24]:
dimensions = 6
p = 5

h = get_hermit(dimensions, p)
cone = apply_restrictor(h, dimensions, p)
# cone = np.ndarray(cone, dtype=float)
# print cone

c = cone[-1]
x = np.zeros(shape=(dimensions,1), dtype=float)
x[-1] = float(dimensions)/float(p)

sigma = c.dot(x)  # ( b[n+1])

f = copy(x)#np.reshape(np.append(x, sigma), newshape=(dimensions + 1, 1))
f[-1] = float(dimensions + 1)/float(p)

# print 'f (vector-indicator`s last value): %s' % f[-1]  #  == sol

if c.dot(f) > sigma:
    print 'f belongs to M={x: cx >= sigma}'
else:
    print 'f doesn`t belong to M={x: cx >= sigma}'
    
target_f = matrix(np.zeros((dimensions + 1,1),dtype=float), tc='d')
target_f[0] = 1

# print target_f
# print matrix(cone, tc='d')
# print matrix(f, tc='d')
# print set(range(dimensions))

(status,s) = ilp(target_f, 
                 G=matrix(-cone, tc='d'),
                 h=matrix(np.append(f, sigma), tc='d'),
#                  A=matrix(1., (0,6)),b=matrix(1., (0,1)),
                 I=set(range(dimensions)),
#                  B=set()
                ) 
print status
if is_valid_solution(s):
    print s
else:
    print 'Trivial solution.'

f belongs to M={x: cx >= sigma}
optimal
Trivial solution.


In [4]:
dims = 3
A = matrix(-get_hermit(dims, 2), tc='d')
c=matrix(np.zeros(dims,dtype=int))
c[0] = 1
for b in generate_b(dims):
#     m = Matrix(np.concatenate((A,b),axis=1))
    (status,s) = ilp(c, G=A, h=matrix(b),I=set(range(dims)))  #  A=None, b=None,, B=None
#     if is_valid_solution(s):
    print(A)
    print(b)        
    print(s)
    print('_____________________\n')

NameError: name 'generate_b' is not defined

In [ ]:
A = get_hermit()
for b in generate_b():
    m = Matrix(np.concatenate((A,b),axis=1))
    s = linsolve(m, symbols('x1 x2 x3 x4 x5 x6'))
    if is_valid_solution(s):
        print(A)
        print(b)        
        print(s)
        print('_____________________\n')

In [ ]:
# #! /usr/bin/python2
# from itertools import chain
# import numpy as np
# from copy import copy
# from numpy.linalg import det 

# def get_hermit(dimensions=5, max_minor=2):
#     diag = [1]*(dimensions)
#     diag[-1] = max_minor
#     pre_last = np.random.randint(max_minor, size=dimensions - 1)
#     hermit = np.append(np.diag(diag), np.random.randint(-max_minor, max_minor + 1, size=(1, dimensions)), 0)
#     hermit[dimensions-1][:-1] = pre_last
#     return hermit


# def minor_mat(arr, i=0):
# #     ith row, jth column removed
#     return arr[np.array(range(i)+range(i+1, arr.shape[0]))[:, np.newaxis],
#                np.array(range(arr.shape[1]))]


# # def minor_mat(arr, i=0, j=0):
#     # ith row, jth column removed
# #     return arr[np.array(range(i)+range(i+1, arr.shape[0]))[:, np.newaxis],
# #                np.array(range(j)+range(j+1, arr.shape[1]))]

# def exclude_row(arr, i=0):
#     # ith row, jth column removed
#     return arr[np.array(chain(range(i),range(i+1, arr.shape[0])))[:, np.newaxis],
#                np.array(range(arr.shape[1]))]


# def get_det_of_minor(hermite):
#     # return [abs(d) for d in hermite: ]
#     for minor_counter in range(hermite.shape[1]):
#         yield abs(det(minor_mat(hermite, i=0, j=minor_counter)))
#         # prime = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]

# h = get_hermit()
# # print(h)
# # print('\n')
# # print(h[chain(range(1,2), range(3,5))])
# # print(exclude_row(h, 0))

# for i in range(h.shape[0]):
#     print(det(minor_mat(h,i)))
# #     m = copy(h[:-1])
# #     print(m)
# #     m[i] = h[-1]
# #     print(m)
#     #h[list(range(i)).extend(list(range(i+2, h.shape[0])))]
# #             np.array(range(h.shape[1]),dtype='int')])
# #     print(m)
# # det(h)


# A = get_hermit()
# print(A)
# print('\n')
# # print(is_valid(A))
# # if is_valid(A):
# for b in generate_b():
#     print(b)
# # print(np.linalg.solve(A, b))



# def exclude_row(arr, i=0):
#     # ith row is removed
#     return arr[np.array(chain(range(i),range(i+1, arr.shape[0])))[:, np.newaxis],
#                np.array(range(arr.shape[1]))]

# def get_det_of_minor(hermite):
#     # return [abs(d) for d in hermite: ]
#     for minor_counter in range(hermite.shape[1]):
#         yield abs(det(minor_mat(hermite, i=0, j=minor_counter)))